In [ ]:
# Installer les packages 

%pip install roboflow 
%pip install pycocotools
%pip install -U albumentations

In [ ]:
# Télecharger notre dataset 
from roboflow import Roboflow

rf = Roboflow(api_key="KhV6y3SVx0M66EFe23Vv")
project = rf.workspace("bootit").project("yolov5-peoples-cars")
version = project.version(8)
dataset = version.download("yolov9")

In [ ]:
# Lancer le script d'augmentations de données (augmentation géométriques et brumes)
!python /kaggle/working/Object-Detection/yolov9/aug_yolov9.py -c /kaggle/working/Object-Detection/yolov9/data_config/augmentation.yaml

In [ ]:
# Entrainer notre modèle pour 30 epochs pour qu'on puisse l'utiliser dans la génération des attaques adversariales
from ultralytics import YOLO

# Télécharger le modèle préentrainé par défault
model = YOLO("yolov9s")

# lancer l'étape d'entrainement
model.train(data="/kaggle/working/Object-Detection/FRCNN/data_configs/ppe.yaml", epochs=30, imgsz=640, batch=64, device =[0,1], patience=20)

In [ ]:
# Lancer le script des attaques adversariales

!python /kaggle/working/Object-Detection/yolov9/fgsm_yolov9.py -c /kaggle/working/Object-Detection/yolov9/data_config/fgsm.yaml

In [ ]:
# Réentainer encore notre modèle sur la base de donnée finale en utilisant le modèle déja obtenu dès la première tentative d'entrainement
model = YOLO("/kaggle/input/yolov9-30-epochs-no-attack-model/best.pt")

model.train(data="/kaggle/working/Object-Detection/FRCNN/data_configs/ppe.yaml", epochs=100, imgsz=640, batch=64, device =[0,1], patience=20, model = '/kaggle/input/yolov9-30-epochs-no-attack-model/best.pt')